![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database
- Schema
- Queries
- Tables
- Views
- Primary Keys
- Foreign Keys
- Indexes
- Functions & User Defined Functions (UDFs)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [4]:
cursor.description

(('name', None, None, None, None, None, None),)

In [5]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [6]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [7]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [8]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [9]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [10]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [11]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [12]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [13]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [14]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [15]:
cursor.execute("""
SELECT latitude, longitude
FROM airports
LEFT JOIN routes ON airports.
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('id', None, None, None, None, None, None)
('name', None, None, None, None, None, None)
('city', None, None, None, None, None, None)
('country', None, None, None, None, None, None)
('code', None, None, None, None, None, None)
('icao', None, None, None, None, None, None)
('latitude', None, None, None, None, None, None)
('longitude', None, None, None, None, None, None)
('altitude', None, None, None, None, None, None)
('offset', None, None, None, None, None, None)
('dst', None, None, None, None, None, None)
('timezone', None, None, None, None, None, None)


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [ ]:
# Which countries have the highest amount of active airports?
cursor.execute("""
-- your code here;
""")
print(*cursor.fetchall(), sep='\n')

In [ ]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
-- your code here;
""")
print(*cursor.fetchall(), sep='\n')

In [ ]:
# What about airports by timezones?
cursor.execute("""
-- your code here;""")
print(*cursor.fetchall(), sep='\n')

In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [16]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [17]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
43,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
290,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
379,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
927,A784016,Max,05/21/2019 03:13:00 PM,05/21/2019 03:13:00 PM,08/10/2018,Return to Owner,NaN,Dog,Intact Male,9 months,Staffordshire Mix,Brown Brindle/White
980,A681062,Max,06/12/2014 02:22:00 PM,06/12/2014 02:22:00 PM,06/11/2011,Return to Owner,NaN,Dog,Neutered Male,3 years,Jack Russell Terrier Mix,Brown/White
1023,A727715,Max,10/07/2016 05:40:00 PM,10/07/2016 05:40:00 PM,05/25/2015,Adoption,NaN,Dog,Neutered Male,1 year,German Shepherd Mix,Tan/Black
1694,A691126,Max,01/06/2016 01:12:00 PM,01/06/2016 01:12:00 PM,10/30/2005,Return to Owner,NaN,Dog,Neutered Male,10 years,Maltese/Miniature Poodle,White
1805,A691271,Max,12/20/2016 08:50:00 AM,12/20/2016 08:50:00 AM,11/03/2013,Transfer,Partner,Dog,Neutered Male,3 years,Chihuahua Shorthair/Finnish Spitz,White/Tan
2655,A795976,Max,05/30/2019 11:20:00 AM,05/30/2019 11:20:00 AM,05/27/2008,Transfer,Partner,Dog,Neutered Male,11 years,Maltese,White
3069,A796351,Max,06/01/2019 04:18:00 PM,06/01/2019 04:18:00 PM,05/31/2010,Return to Owner,NaN,Cat,Neutered Male,9 years,Domestic Shorthair,Black/White


In [18]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A789027,Lennie,02/17/2019 11:44:00 AM,02/17/2019 11:44:00 AM,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream
1,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
9,A789027,Lennie,03/10/2019 12:25:00 PM,03/10/2019 12:25:00 PM,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream
12,A795059,NaN,05/16/2019 10:14:00 AM,05/16/2019 10:14:00 AM,03/15/2019,Euthanasia,Rabies Risk,Dog,Intact Male,2 months,Pit Bull,Brown Brindle/White
13,A795043,NaN,05/16/2019 10:17:00 AM,05/16/2019 10:17:00 AM,03/31/2019,Euthanasia,Rabies Risk,Dog,Intact Male,1 month,German Shepherd,Black/Tan


## Goal 6: Transfering from sqlite to pandas

In [19]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [ ]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
-- your code here;
""", pd_con)
df

In [ ]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [ ]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [ ]:
# How many tables are in the database?

# your code here

In [ ]:
# What's the primary key of each table?

# your code here

In [ ]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
-- your code here;
""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"


In [ ]:
# What values from each table?


In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

## Reflection

## Learning goals review:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1O0dMGqjIV6-JIDYSIeGqTjIdep1BFiIVCc29UbEgwUM/edit)!